<div style="background-color: #04D7FD; padding: 20px; text-align: left;">
    <h1 style="color: #000000; font-size: 36px; margin: 0;">Data Processing for RAG with Data Prep Kit (RAY)</h1>
    
</div>


## Before Running the notebook

Please complete [setting up python dev environment](./setup-python-dev-env.md)

## Overview

This notebook will process PDF documents as part of RAG pipeline

![](media/rag-overview-2.png)

This notebook will perform steps 1, 2 and 3 in RAG pipeline.

Here are the processing steps:

- **pdf2parquet** : Extract text from PDF and convert them into parquet files
- **Chunk documents**: Split the PDFs into 'meaningful sections' (paragraphs, sentences ..etc)
- **Doc_ID generation**: Each chunk is assigned a uniq id, based on content and hash
- **Exact Dedup**: Chunks with exact same content are filtered out
- **Fuzzy Dedup**: Eliminate chunks that are 'very similar' content
- **Doc quality**: Scores the documents based on criteria like number of words, if it contains bad words ..etc
- **Text encoder**: Convert chunks into vectors using embedding models

## Step-1: Configuration

### 1.1 - Common Config

In [1]:
from my_config import MY_CONFIG


### 1.2 - Inspect Input Data

We have a bnunch of datasets in [data](../data) folder.  Examine them

We will use one of them or feel free to bring your own data.

### 1.3  - Set input/output path variables for the pipeline

In [2]:
import os, sys
import shutil

if not os.path.exists(MY_CONFIG.INPUT_DATA_DIR ):
    raise Exception (f"❌ Input folder MY_CONFIG.INPUT_DATA_DIR = '{MY_CONFIG.INPUT_DATA_DIR}' not found")

output_parquet_dir = os.path.join (MY_CONFIG.OUTPUT_FOLDER, '01_parquet_out')
output_chunk_dir = os.path.join (MY_CONFIG.OUTPUT_FOLDER, '02_chunk_out')
output_docid_dir = os.path.join (MY_CONFIG.OUTPUT_FOLDER, '03_docid_out')
output_exact_dedupe_dir = os.path.join (MY_CONFIG.OUTPUT_FOLDER, '04_exact_dedupe_out')
output_fuzzy_dedupe_dir = os.path.join (MY_CONFIG.OUTPUT_FOLDER, '05_fuzzy_dedupe_out')
output_embeddings_dir = os.path.join (MY_CONFIG.OUTPUT_FOLDER, '06_embeddings_out')


## clear output folder
shutil.rmtree(MY_CONFIG.OUTPUT_FOLDER, ignore_errors=True)
shutil.os.makedirs(MY_CONFIG.OUTPUT_FOLDER, exist_ok=True)

print ("✅ Cleared output directory")

✅ Cleared output directory


### 1.4 - Import Common python modules

In [3]:
import os
import sys


from data_processing_ray.runtime.ray import RayTransformLauncher
from data_processing.utils import ParamsUtils

<a id="pdf2parquet"></a>

## Step-2: pdf2parquet -  Convert data from PDF to Parquet

This step is reading the input folder containing all PDF files and ingest them in a parquet table using the [Docling package](https://github.com/DS4SD/docling).
The documents are converted into a JSON format which allows to easily chunk it in the later steps.



### 2.1 - Set Input/output Folder

In [4]:
STAGE = 1 

input_folder = MY_CONFIG.INPUT_DATA_DIR
output_folder =  output_parquet_dir

print (f"🏃🏼 STAGE-{STAGE}: Processing input='{input_folder}' --> output='{output_folder}'")

🏃🏼 STAGE-1: Processing input='../data/solar-system' --> output='output-solar-system/01_parquet_out'


### 2.2 -  Execute 

In [5]:
%%time 

import ast
import os
import sys

from pdf2parquet_transform import (
    pdf2parquet_contents_type_cli_param,
    pdf2parquet_contents_types,
)
from pdf2parquet_transform_python import Pdf2ParquetPythonTransformConfiguration
from pdf2parquet_transform_ray import Pdf2ParquetRayTransformConfiguration

from data_processing.utils import GB, ParamsUtils


# create parameters
local_conf = {
    "input_folder": input_folder,
    "output_folder": output_folder,
}
worker_options = {"num_cpus" : MY_CONFIG.RAY_NUM_CPUS, "memory": MY_CONFIG.RAY_MEMORY_GB * GB}
code_location = {"github": "github", "commit_hash": "12345", "path": "path"}
ingest_config = {
    pdf2parquet_contents_type_cli_param: pdf2parquet_contents_types.JSON,
}

params = {
    # where to run
    "run_locally": True,
    # Data access. Only required parameters are specified
    "data_local_config": ParamsUtils.convert_to_ast(local_conf),
    "data_files_to_use": ast.literal_eval("['.pdf']"),
    # orchestrator
    "runtime_worker_options": ParamsUtils.convert_to_ast(worker_options),
    "runtime_num_workers": MY_CONFIG.RAY_RUNTIME_WORKERS,
    "runtime_pipeline_id": "pipeline_id",
    "runtime_job_id": "job_id",
    "runtime_code_location": ParamsUtils.convert_to_ast(code_location),
}


sys.argv = ParamsUtils.dict_to_req(d=(params | ingest_config))
# create launcher
launcher = RayTransformLauncher(Pdf2ParquetRayTransformConfiguration())
# launch
return_code = launcher.launch()

if return_code == 0:
    print (f"✅ Stage:{STAGE} completed successfully")
else:
    raise Exception ("❌ Ray job failed")


11:49:07 INFO - Running locally
11:49:07 INFO - pdf2parquet parameters are : {'artifacts_path': None, 'contents_type': <pdf2parquet_contents_types.JSON: 'application/json'>, 'do_table_structure': True, 'do_ocr': True, 'double_precision': 8}
11:49:07 INFO - data factory data_ is using local data access: input_folder - ../data/solar-system output_folder - output-solar-system/01_parquet_out
11:49:07 INFO - data factory data_ max_files -1, n_sample -1
11:49:07 INFO - data factory data_ Not using data sets, checkpointing False, max files -1, random samples -1, files to use ['.pdf'], files to checkpoint ['.parquet']
11:49:07 INFO - pipeline id pipeline_id
11:49:07 INFO - code location {'github': 'github', 'commit_hash': '12345', 'path': 'path'}
11:49:07 INFO - number of workers 2 worker options {'num_cpus': 1, 'memory': 2147483648, 'max_restarts': -1}
11:49:07 INFO - actor creation delay 0
11:49:07 INFO - job details {'job category': 'preprocessing', 'job name': 'pdf2parquet', 'job type': 'r

✅ Stage:1 completed successfully
CPU times: user 3.48 s, sys: 500 ms, total: 3.98 s
Wall time: 27.4 s


### 2.3 - Inspect Generated output

Here we should see one entry per input file processed

In [6]:
from utils import read_parquet_files_as_df

output_df = read_parquet_files_as_df(output_folder)

print ("Output dimensions (rows x columns)= ", output_df.shape)

output_df.head(5)

## To display certain columns
#parquet_df[['column1', 'column2', 'column3']].head(5)

Output dimensions (rows x columns)=  (2, 12)


,filename,contents,num_pages,num_tables,num_doc_elements,document_id,ext,hash,size,date_acquired,pdf_convert_time,source_filename
0,mars.pdf,"{""_name"":"""",""type"":""pdf-document"",""description...",1,0,11,39513892-bd6e-4935-b196-ef95ea5804fa,pdf,8edd5dfbf888777120b528a5d8998f2757d006df0eaef7...,2800,2024-09-20T11:49:20.974002,1.919181,mars.pdf
1,earth.pdf,"{""_name"":"""",""type"":""pdf-document"",""description...",1,0,11,2573f862-f4a5-4d92-b8b9-6ff8cc92b8f0,pdf,18713f970989055625bef22209b6f4b6830b9ca22046bf...,2686,2024-09-20T11:49:21.020310,1.936369,earth.pdf


<a id="chunking"></a>

##  Step-3: Doc chunks

Split the documents in chunks, according to their layout segmentation.

### 3.1 - Set Input/output Folder

In [7]:
STAGE = 2

input_folder = output_parquet_dir # previous output folder is the input folder for the current stage
output_folder =  output_chunk_dir

input_df = read_parquet_files_as_df(input_folder)  ## for debug purposes

print (f"🏃🏼 STAGE-{STAGE}: Processing input='{input_folder}' --> output='{output_folder}'")

🏃🏼 STAGE-2: Processing input='output-solar-system/01_parquet_out' --> output='output-solar-system/02_chunk_out'


### 3.2 -  Execute 

In [8]:
%%time 

# Import doc_json_chunk transform configuration
from doc_chunk_transform_ray import DocChunkRayTransformConfiguration


# Prepare the commandline params
local_conf = {
    "input_folder": input_folder,
    "output_folder": output_folder,
}
worker_options = {"num_cpus" : MY_CONFIG.RAY_NUM_CPUS}
params = {
    # where to run
    "run_locally": True,
    # Data access. Only required parameters are specified
    "data_local_config": ParamsUtils.convert_to_ast(local_conf),
    # orchestrator
    "runtime_worker_options": ParamsUtils.convert_to_ast(worker_options),
    "runtime_num_workers": MY_CONFIG.RAY_RUNTIME_WORKERS,
    # doc_chunk arguments
    # ...
}

# Pass the commandline params
sys.argv = ParamsUtils.dict_to_req(d=params)

# create launcher
launcher = RayTransformLauncher(DocChunkRayTransformConfiguration())
# launch
return_code = launcher.launch()

if return_code == 0:
    print (f"✅ Stage:{STAGE} completed successfully")
else:
    raise Exception ("❌ Ray job failed")

11:49:33 INFO - Running locally
11:49:33 INFO - doc_chunk parameters are : {'chunking_type': <chunking_types.DL_JSON: 'dl_json'>, 'content_column_name': 'contents', 'output_chunk_column_name': 'contents', 'output_jsonpath_column_name': 'doc_jsonpath', 'output_pageno_column_name': 'page_number', 'output_bbox_column_name': 'bbox'}
11:49:33 INFO - data factory data_ is using local data access: input_folder - output-solar-system/01_parquet_out output_folder - output-solar-system/02_chunk_out
11:49:33 INFO - data factory data_ max_files -1, n_sample -1
11:49:33 INFO - data factory data_ Not using data sets, checkpointing False, max files -1, random samples -1, files to use ['.parquet'], files to checkpoint ['.parquet']
11:49:33 INFO - pipeline id pipeline_id
11:49:33 INFO - code location None
11:49:33 INFO - number of workers 2 worker options {'num_cpus': 1, 'max_restarts': -1}
11:49:33 INFO - actor creation delay 0
11:49:33 INFO - job details {'job category': 'preprocessing', 'job name': '

✅ Stage:2 completed successfully
CPU times: user 915 ms, sys: 275 ms, total: 1.19 s
Wall time: 18.4 s


### 3.3 - Inspect Generated output

We would see documents are split into many chunks

In [9]:
from utils import read_parquet_files_as_df

output_df = read_parquet_files_as_df(output_folder)

print (f"Files processed : {input_df.shape[0]:,}")
print (f"Chunks created : {output_df.shape[0]:,}")

print ("Input data dimensions (rows x columns)= ", input_df.shape)
print ("Output data dimensions (rows x columns)= ", output_df.shape)

output_df.sample(min(3, output_df.shape[0]))

Files processed : 2
Chunks created : 8
Input data dimensions (rows x columns)=  (2, 12)
Output data dimensions (rows x columns)=  (8, 15)


,filename,num_pages,num_tables,num_doc_elements,document_id,ext,hash,size,date_acquired,pdf_convert_time,source_filename,contents,doc_jsonpath,page_number,bbox
3,mars.pdf,1,0,11,39513892-bd6e-4935-b196-ef95ea5804fa,pdf,8edd5dfbf888777120b528a5d8998f2757d006df0eaef7...,2800,2024-09-20T11:49:20.974002,1.919181,mars.pdf,Basic facts about Mars:\n· Distance from the S...,$.main-text[6],1,"[133.2026062, 482.90710449, 237.04431152, 493...."
6,earth.pdf,1,0,11,2573f862-f4a5-4d92-b8b9-6ff8cc92b8f0,pdf,18713f970989055625bef22209b6f4b6830b9ca22046bf...,2686,2024-09-20T11:49:21.020310,1.936369,earth.pdf,Earth\nEarth is the third planet from the Sun....,$.main-text[5],1,"[132.91053772, 512.46295166, 477.84887695, 534..."
1,mars.pdf,1,0,11,39513892-bd6e-4935-b196-ef95ea5804fa,pdf,8edd5dfbf888777120b528a5d8998f2757d006df0eaef7...,2800,2024-09-20T11:49:20.974002,1.919181,mars.pdf,Solar System\nFor more details about the Solar...,$.main-text[3],1,"[133.18510437, 570.83258057, 374.99838257, 581..."


## Step-4:  DOC ID generation

This transform annotates documents with document "ids". It supports the following transformations of the original data:

 - Adding document hash: this enables the addition of a document hash-based id to the data. The hash is calculated with `hashlib.sha256(doc.encode("utf-8")).hexdigest()`. To enable this annotation, set hash_column to the name of the column, where you want to store it.
 - Adding integer document id: this allows the addition of an integer document id to the data that is unique across all rows in all tables provided to the transform() method. To enable this annotation, set int_id_column to the name of the column, where you want to store it. **This is a pre-requisite for fuzzy dedup** in the pipeline.

### 4.1 - Set Input/output Folder

In [10]:

STAGE  = 3

input_folder = output_chunk_dir # previous output folder is the input folder for the current stage
output_folder =  output_docid_dir

input_df = read_parquet_files_as_df(input_folder)  ## for debug purposes

print (f"🏃🏼 STAGE-{STAGE}: Processing input='{input_folder}' --> output='{output_folder}'")

🏃🏼 STAGE-3: Processing input='output-solar-system/02_chunk_out' --> output='output-solar-system/03_docid_out'


### 4.2 - Execute 

In [11]:
%%time 

from doc_id_transform_ray import DocIDRayTransformRuntimeConfiguration
local_conf = {
    "input_folder": input_folder,
    "output_folder": output_folder,
}
worker_options = {"num_cpus" : MY_CONFIG.RAY_NUM_CPUS}
params = {
    # where to run
    "run_locally": True,
    # Data access. Only required parameters are specified
    "data_local_config": ParamsUtils.convert_to_ast(local_conf),
    # orchestrator
    "runtime_worker_options": ParamsUtils.convert_to_ast(worker_options),
    "runtime_num_workers": MY_CONFIG.RAY_RUNTIME_WORKERS,
    # doc id configuration
    "doc_id_doc_column": "contents",
    "doc_id_hash_column": "chunk_hash",
    "doc_id_int_column": "chunk_id",
}
sys.argv = ParamsUtils.dict_to_req(d=params)

# launch

launcher = RayTransformLauncher(DocIDRayTransformRuntimeConfiguration())

return_code = launcher.launch()

if return_code == 0:
    print (f"✅ Stage:{STAGE} completed successfully")
else:
    raise Exception ("❌ Ray job failed")

11:49:50 INFO - Running locally
11:49:50 INFO - Doc id parameters are : {'doc_column': 'contents', 'hash_column': 'chunk_hash', 'int_column': 'chunk_id', 'start_id': 0}
11:49:50 INFO - data factory data_ is using local data access: input_folder - output-solar-system/02_chunk_out output_folder - output-solar-system/03_docid_out
11:49:50 INFO - data factory data_ max_files -1, n_sample -1
11:49:50 INFO - data factory data_ Not using data sets, checkpointing False, max files -1, random samples -1, files to use ['.parquet'], files to checkpoint ['.parquet']
11:49:50 INFO - pipeline id pipeline_id
11:49:50 INFO - code location None
11:49:50 INFO - number of workers 2 worker options {'num_cpus': 1, 'max_restarts': -1}
11:49:50 INFO - actor creation delay 0
11:49:50 INFO - job details {'job category': 'preprocessing', 'job name': 'doc_id', 'job type': 'ray', 'job id': 'job_id'}
2024-09-20 11:49:52,750	INFO worker.py:1744 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:

✅ Stage:3 completed successfully
CPU times: user 127 ms, sys: 166 ms, total: 292 ms
Wall time: 15.1 s


### 4.3 - Inspect Generated output

In [12]:
from utils import read_parquet_files_as_df

output_df = read_parquet_files_as_df(output_folder)

print ("Input data dimensions (rows x columns)= ", input_df.shape)
print ("Output data dimensions (rows x columns)= ", output_df.shape)

output_df.sample(min(3, output_df.shape[0]))

Input data dimensions (rows x columns)=  (8, 15)
Output data dimensions (rows x columns)=  (8, 17)


,filename,num_pages,num_tables,num_doc_elements,document_id,ext,hash,size,date_acquired,pdf_convert_time,source_filename,contents,doc_jsonpath,page_number,bbox,chunk_hash,chunk_id
1,mars.pdf,1,0,11,39513892-bd6e-4935-b196-ef95ea5804fa,pdf,8edd5dfbf888777120b528a5d8998f2757d006df0eaef7...,2800,2024-09-20T11:49:20.974002,1.919181,mars.pdf,Solar System\nFor more details about the Solar...,$.main-text[3],1,"[133.18510437, 570.83258057, 374.99838257, 581...",dee4c03474c98efdabbadbcc4ce91138c7820f4ac8ff07...,1
3,mars.pdf,1,0,11,39513892-bd6e-4935-b196-ef95ea5804fa,pdf,8edd5dfbf888777120b528a5d8998f2757d006df0eaef7...,2800,2024-09-20T11:49:20.974002,1.919181,mars.pdf,Basic facts about Mars:\n· Distance from the S...,$.main-text[6],1,"[133.2026062, 482.90710449, 237.04431152, 493....",7ff317954ec5f3b15607c053c30c2b0db0f6b64cc3295a...,3
6,earth.pdf,1,0,11,2573f862-f4a5-4d92-b8b9-6ff8cc92b8f0,pdf,18713f970989055625bef22209b6f4b6830b9ca22046bf...,2686,2024-09-20T11:49:21.020310,1.936369,earth.pdf,Earth\nEarth is the third planet from the Sun....,$.main-text[5],1,"[132.91053772, 512.46295166, 477.84887695, 534...",7c4a750e2215f231803a6f8078bde1e9699034fb033dd3...,6


## Step-5: Exact Dedup

Remove documents having identical code to remove bias in the training data. On the content of each document, a SHA256 hash is computed,
followed by de-duplication of record having identical hashes.

### 5.1 -  Set Input/output Folder

In [13]:
STAGE  = 4

input_folder = output_docid_dir # previous output folder is the input folder for the current stage
output_folder =  output_exact_dedupe_dir

input_df = read_parquet_files_as_df(input_folder)  ## for debug purposes

print (f"🏃🏼 STAGE-{STAGE}: Processing input='{input_folder}' --> output='{output_folder}'")

🏃🏼 STAGE-4: Processing input='output-solar-system/03_docid_out' --> output='output-solar-system/04_exact_dedupe_out'


### 5.2 - Execute 

In [14]:
%%time

# Import ededup transform configuration
from ededup_transform_ray import EdedupRayTransformRuntimeConfiguration


# Prepare the commandline params
local_conf = {
    "input_folder": input_folder,
    "output_folder": output_folder,
}
worker_options = {"num_cpus" : MY_CONFIG.RAY_NUM_CPUS}
params = {
    # where to run
    "run_locally": True,
    # Data access. Only required parameters are specified
    "data_local_config": ParamsUtils.convert_to_ast(local_conf),
    # orchestrator
    "runtime_worker_options": ParamsUtils.convert_to_ast(worker_options),
    "runtime_num_workers": MY_CONFIG.RAY_RUNTIME_WORKERS,
    # ededup parameters
    "ededup_hash_cpu": 0.5,
    "ededup_num_hashes": 2,
    "ededup_doc_column": "contents",
    "ededup_doc_id_column": "chunk_hash",
    
}

# Pass the commandline params
sys.argv = ParamsUtils.dict_to_req(d=params)

# create launcher
launcher = RayTransformLauncher(EdedupRayTransformRuntimeConfiguration())
# launch
return_code = launcher.launch()

if return_code == 0:
    print (f"✅ Stage:{STAGE} completed successfully")
else:
    raise Exception ("❌ Ray job failed")

11:50:06 INFO - Running locally
11:50:06 INFO - exact dedup params are {'doc_column': 'contents', 'doc_id_column': 'chunk_hash', 'use_snapshot': False, 'snapshot_directory': None, 'hash_cpu': 0.5, 'num_hashes': 2}
11:50:06 INFO - data factory data_ is using local data access: input_folder - output-solar-system/03_docid_out output_folder - output-solar-system/04_exact_dedupe_out
11:50:06 INFO - data factory data_ max_files -1, n_sample -1
11:50:06 INFO - data factory data_ Not using data sets, checkpointing False, max files -1, random samples -1, files to use ['.parquet'], files to checkpoint ['.parquet']
11:50:06 INFO - pipeline id pipeline_id
11:50:06 INFO - code location None
11:50:06 INFO - number of workers 2 worker options {'num_cpus': 1, 'max_restarts': -1}
11:50:06 INFO - actor creation delay 0
11:50:06 INFO - job details {'job category': 'preprocessing', 'job name': 'ededup', 'job type': 'ray', 'job id': 'job_id'}
2024-09-20 11:50:07,944	INFO worker.py:1744 -- Started a local R

✅ Stage:4 completed successfully
CPU times: user 125 ms, sys: 152 ms, total: 277 ms
Wall time: 15.3 s


### 5.3 - Inspect Generated output

In [15]:
from utils import read_parquet_files_as_df

output_df = read_parquet_files_as_df(output_folder)

print ("Input data dimensions (rows x columns)= ", input_df.shape)
print ("Output data dimensions (rows x columns)= ", output_df.shape)
print (f"Input chunks before exact dedupe : {input_df.shape[0]:,}")
print (f"Output chunks after exact dedupe : {output_df.shape[0]:,}")
print ("Duplicate chunks removed :  ", (input_df.shape[0] - output_df.shape[0]))

output_df.sample(min(3, output_df.shape[0]))

Input data dimensions (rows x columns)=  (8, 17)
Output data dimensions (rows x columns)=  (7, 18)
Input chunks before exact dedupe : 8
Output chunks after exact dedupe : 7
Duplicate chunks removed :   1


,filename,num_pages,num_tables,num_doc_elements,document_id,ext,hash,size,date_acquired,pdf_convert_time,source_filename,contents,doc_jsonpath,page_number,bbox,chunk_hash,chunk_id,removed
5,earth.pdf,1,0,11,2573f862-f4a5-4d92-b8b9-6ff8cc92b8f0,pdf,18713f970989055625bef22209b6f4b6830b9ca22046bf...,2686,2024-09-20T11:49:21.020310,1.936369,earth.pdf,Earth\nEarth is the third planet from the Sun....,$.main-text[5],1,"[132.91053772, 512.46295166, 477.84887695, 534...",7c4a750e2215f231803a6f8078bde1e9699034fb033dd3...,6,[]
1,mars.pdf,1,0,11,39513892-bd6e-4935-b196-ef95ea5804fa,pdf,8edd5dfbf888777120b528a5d8998f2757d006df0eaef7...,2800,2024-09-20T11:49:20.974002,1.919181,mars.pdf,Solar System\nFor more details about the Solar...,$.main-text[3],1,"[133.18510437, 570.83258057, 374.99838257, 581...",dee4c03474c98efdabbadbcc4ce91138c7820f4ac8ff07...,1,[]
6,earth.pdf,1,0,11,2573f862-f4a5-4d92-b8b9-6ff8cc92b8f0,pdf,18713f970989055625bef22209b6f4b6830b9ca22046bf...,2686,2024-09-20T11:49:21.020310,1.936369,earth.pdf,Earth\nBasic facts about Earth:\n· Distance fr...,$.main-text[6],1,"[133.30151367, 494.86206055, 240.17156982, 505...",189a221704d17feeb96b1b1ef60a2a2445459848cd8e8f...,7,[]


## Step-6: Fuzzy Dedup

Post exact deduplication, fuzzy deduplication is applied with
the goal of removing code files that may have slight variations and thereby unbiasing
the data further. Small variations are quite commonly seen in code data in the form
of variations in the values of variables, addittion of logging statements etc. Find near-
duplicate.

### 6.1 - Set Input/output Folder

In [16]:
## Input to this component is the output of doc_id generator component. 

STAGE  = 5

input_folder = output_exact_dedupe_dir # previous output folder is the input folder for the current stage
output_folder =  output_fuzzy_dedupe_dir

input_df = read_parquet_files_as_df(input_folder)  ## for debug purposes

print (f"🏃🏼 STAGE-{STAGE}: Processing input='{input_folder}' --> output='{output_folder}'")

🏃🏼 STAGE-5: Processing input='output-solar-system/04_exact_dedupe_out' --> output='output-solar-system/05_fuzzy_dedupe_out'


### 6.2 - Execute 

In [17]:
%%time 

import os
import sys

from data_processing.utils import ParamsUtils
from fdedup_transform_ray import FdedupRayTransformConfiguration

# create parameters

local_conf = {
    "input_folder": input_folder,
    "output_folder": output_folder,
}
worker_options = {"num_cpus" : MY_CONFIG.RAY_NUM_CPUS}
code_location = {"github": "github", "commit_hash": "12345", "path": "path"}
params = {
    # where to run
    "run_locally": True,
    # Data access. Only required parameters are specified
    "data_local_config": ParamsUtils.convert_to_ast(local_conf),
    # Orchestration parameters
    "runtime_worker_options": ParamsUtils.convert_to_ast(worker_options),
    "runtime_num_workers": MY_CONFIG.RAY_RUNTIME_WORKERS,
    # columns used
    "fdedup_doc_column": "contents",
    "fdedup_id_column": "chunk_id",
    "fdedup_cluster_column": "chunk_hash",
    # infrastructure
    "fdedup_bucket_cpu": 0.3,
    "fdedup_doc_cpu": 0.3,
    "fdedup_mhash_cpu": 0.3,
    "fdedup_num_doc_actors": 1,
    "fdedup_num_bucket_actors": 1,
    "fdedup_num_minhash_actors": 1,
    "fdedup_num_preprocessors": 1,
    # fuzzy parameters
    "fdedup_num_permutations": 64,
    "fdedup_threshold": 0.7, # between 0.0 to 1.0 ; smaller values tend to be more lenient in finding near dupes; close to 1.0 is more strict
    "fdedup_shingles_size": 5,
    "fdedup_delimiters": " "
}

# Pass commandline params
sys.argv = ParamsUtils.dict_to_req(d=params)

# launch

launcher = RayTransformLauncher(FdedupRayTransformConfiguration())

return_code = launcher.launch()

if return_code == 0:
    print (f"✅ Stage:{STAGE} completed successfully")
else:
    raise Exception ("❌ Ray job failed")

11:50:21 INFO - Running locally
11:50:21 INFO - fuzzy dedup params are {'doc_column': 'contents', 'id_column': 'chunk_id', 'cluster_column': 'chunk_hash', 'bucket_cpu': 0.3, 'mhash_cpu': 0.3, 'doc_cpu': 0.3, 'num_doc_actors': 1, 'num_minhash_actors': 1, 'num_bucket_actors': 1, 'num_preprocessors': 1, 'num_permutations': 64, 'threshold': 0.7, 'shingles_size': 5, 'delimiters': ' ', 'snapshot_delay': 1, 'use_bucket_snapshot': False, 'use_doc_snapshot': False, 'random_delay_limit': 10, 'worker_options': {'num_cpus': 1}}
11:50:21 INFO - data factory data_ is using local data access: input_folder - output-solar-system/04_exact_dedupe_out output_folder - output-solar-system/05_fuzzy_dedupe_out
11:50:21 INFO - data factory data_ max_files -1, n_sample -1
11:50:21 INFO - data factory data_ Not using data sets, checkpointing False, max files -1, random samples -1, files to use ['.parquet'], files to checkpoint ['.parquet']
11:50:21 INFO - pipeline id pipeline_id
11:50:21 INFO - code location Non

✅ Stage:5 completed successfully
CPU times: user 185 ms, sys: 200 ms, total: 385 ms
Wall time: 36.8 s


### 6.3 - Inspect Generated output

In [18]:
from utils import read_parquet_files_as_df

output_df = read_parquet_files_as_df(output_folder)

print ("Input data dimensions (rows x columns)= ", input_df.shape)
print ("Output data dimensions (rows x columns)= ", output_df.shape)
print ("Duplicate chunks removed  by fuzzy-dedupe:  ", (input_df.shape[0] - output_df.shape[0]))

output_df.sample(min(3, output_df.shape[0]))

Input data dimensions (rows x columns)=  (7, 18)
Output data dimensions (rows x columns)=  (6, 18)
Duplicate chunks removed  by fuzzy-dedupe:   1


,filename,num_pages,num_tables,num_doc_elements,document_id,ext,hash,size,date_acquired,pdf_convert_time,source_filename,contents,doc_jsonpath,page_number,bbox,chunk_id,removed,chunk_hash
0,mars.pdf,1,0,11,39513892-bd6e-4935-b196-ef95ea5804fa,pdf,8edd5dfbf888777120b528a5d8998f2757d006df0eaef7...,2800,2024-09-20T11:49:20.974002,1.919181,mars.pdf,Solar System\nOur solar system is a vast and f...,$.main-text[2],1,"[132.84518433, 588.96014404, 479.40917969, 623...",0,[],-1
3,mars.pdf,1,0,11,39513892-bd6e-4935-b196-ef95ea5804fa,pdf,8edd5dfbf888777120b528a5d8998f2757d006df0eaef7...,2800,2024-09-20T11:49:20.974002,1.919181,mars.pdf,Basic facts about Mars:\n· Distance from the S...,$.main-text[6],1,"[133.2026062, 482.90710449, 237.04431152, 493....",3,[],-1
4,earth.pdf,1,0,11,2573f862-f4a5-4d92-b8b9-6ff8cc92b8f0,pdf,18713f970989055625bef22209b6f4b6830b9ca22046bf...,2686,2024-09-20T11:49:21.020310,1.936369,earth.pdf,Earth\nEarth is the third planet from the Sun....,$.main-text[5],1,"[132.91053772, 512.46295166, 477.84887695, 534...",6,[],-1


## Step-7:   Text encoding

Encode text for the vector storage.

### 7.1 - Set Input/output Folder

In [19]:
STAGE  = 6

input_folder = output_fuzzy_dedupe_dir
output_folder =  output_embeddings_dir

input_df = read_parquet_files_as_df(input_folder)  ## for debug purposes

print (f"🏃🏼 STAGE-{STAGE}: Processing input='{input_folder}' --> output='{output_folder}'")

🏃🏼 STAGE-6: Processing input='output-solar-system/05_fuzzy_dedupe_out' --> output='output-solar-system/06_embeddings_out'


### 7.2 - Execute

In [20]:
%%time 

from text_encoder_transform_ray import TextEncoderRayTransformConfiguration

local_conf = {
    "input_folder": input_folder,
    "output_folder": output_folder,
}
worker_options = {"num_cpus" : MY_CONFIG.RAY_NUM_CPUS}
params = {
    # where to run
    "run_locally": True,
    # Data access. Only required parameters are specified
    "data_local_config": ParamsUtils.convert_to_ast(local_conf),
    # orchestrator
    "runtime_worker_options": ParamsUtils.convert_to_ast(worker_options),
    "runtime_num_workers": MY_CONFIG.RAY_RUNTIME_WORKERS,
    # text_encoder
    "text_encoder_model_name": MY_CONFIG.EMBEDDING_MODEL,
}

sys.argv = ParamsUtils.dict_to_req(d=params)
# create launcher
launcher = RayTransformLauncher(TextEncoderRayTransformConfiguration())
# Launch the ray actor(s) to process the input

return_code = launcher.launch()

if return_code == 0:
    print (f"✅ Stage:{STAGE} completed successfully")
else:
    raise Exception ("❌ Ray job failed")

11:50:58 INFO - Running locally
11:50:58 INFO - text_encoder parameters are : {'content_column_name': 'contents', 'output_embeddings_column_name': 'embeddings', 'model_name': 'sentence-transformers/all-MiniLM-L6-v2'}
11:50:58 INFO - data factory data_ is using local data access: input_folder - output-solar-system/05_fuzzy_dedupe_out output_folder - output-solar-system/06_embeddings_out
11:50:58 INFO - data factory data_ max_files -1, n_sample -1
11:50:58 INFO - data factory data_ Not using data sets, checkpointing False, max files -1, random samples -1, files to use ['.parquet'], files to checkpoint ['.parquet']
11:50:58 INFO - pipeline id pipeline_id
11:50:58 INFO - code location None
11:50:58 INFO - number of workers 2 worker options {'num_cpus': 1, 'max_restarts': -1}
11:50:58 INFO - actor creation delay 0
11:50:58 INFO - job details {'job category': 'preprocessing', 'job name': 'text_encoder', 'job type': 'ray', 'job id': 'job_id'}
2024-09-20 11:51:00,481	INFO worker.py:1744 -- Sta

✅ Stage:6 completed successfully
CPU times: user 456 ms, sys: 204 ms, total: 660 ms
Wall time: 22.6 s


### 7.3 - Inspect Generated output

In [21]:
from utils import read_parquet_files_as_df

output_df = read_parquet_files_as_df(output_folder)

print ("Input data dimensions (rows x columns)= ", input_df.shape)
print ("Output data dimensions (rows x columns)= ", output_df.shape)

output_df.sample(min(3, output_df.shape[0]))

Input data dimensions (rows x columns)=  (6, 18)
Output data dimensions (rows x columns)=  (6, 19)


,filename,num_pages,num_tables,num_doc_elements,document_id,ext,hash,size,date_acquired,pdf_convert_time,source_filename,contents,doc_jsonpath,page_number,bbox,chunk_id,removed,chunk_hash,embeddings
3,mars.pdf,1,0,11,39513892-bd6e-4935-b196-ef95ea5804fa,pdf,8edd5dfbf888777120b528a5d8998f2757d006df0eaef7...,2800,2024-09-20T11:49:20.974002,1.919181,mars.pdf,Basic facts about Mars:\n· Distance from the S...,$.main-text[6],1,"[133.2026062, 482.90710449, 237.04431152, 493....",3,[],-1,"[0.1059802, 0.025460616, 0.02362733, 0.0390564..."
4,earth.pdf,1,0,11,2573f862-f4a5-4d92-b8b9-6ff8cc92b8f0,pdf,18713f970989055625bef22209b6f4b6830b9ca22046bf...,2686,2024-09-20T11:49:21.020310,1.936369,earth.pdf,Earth\nEarth is the third planet from the Sun....,$.main-text[5],1,"[132.91053772, 512.46295166, 477.84887695, 534...",6,[],-1,"[0.0724358, -0.058001805, -0.01977186, -0.0243..."
0,mars.pdf,1,0,11,39513892-bd6e-4935-b196-ef95ea5804fa,pdf,8edd5dfbf888777120b528a5d8998f2757d006df0eaef7...,2800,2024-09-20T11:49:20.974002,1.919181,mars.pdf,Solar System\nOur solar system is a vast and f...,$.main-text[2],1,"[132.84518433, 588.96014404, 479.40917969, 623...",0,[],-1,"[0.0077404897, -0.020559434, 0.026426662, 0.01..."


## Step-8: Copy output to final output dir

In [22]:
import shutil

shutil.rmtree(MY_CONFIG.OUTPUT_FOLDER_FINAL, ignore_errors=True)
shutil.copytree(src=output_folder, dst=MY_CONFIG.OUTPUT_FOLDER_FINAL)

print (f"✅ Copied output from '{output_folder}' --> '{MY_CONFIG.OUTPUT_FOLDER_FINAL}'")

✅ Copied output from 'output-solar-system/06_embeddings_out' --> 'output-solar-system/output_final'
